In [1]:
apikey ='---api---'
url = '---url---'

In [2]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

In [3]:
df = pd.read_csv('covidTweets.csv')

In [4]:
from DataPreprocessing.RemovePuctuations import *
df['text'] = remove_punc_apply(df['text'])
df = df.drop(['Unnamed: 0.1','Unnamed: 0','Unnamed: 14','Unnamed: 15','medias','has_media','user_id','url','ID'], axis=1)

In [5]:
df.head()

,usernameTweet,text,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet
0,rj_ashwin,Sabse Pehle toh Majdur Labour Class Worker...,0,1,0,2020-03-28 23:24:57,False,False
1,vikramsoni232,TajinderBagga Bhaiya i proud of you You a...,0,0,0,2020-03-29 12:52:33,False,False
2,ABHISHE33329022,Mam India mein hum ko companys ki audit krni...,0,0,0,2020-03-29 21:20:30,True,False
3,hindust74474485,Videsh me rahne wale Bhai India laye Gaye bu...,0,0,0,2020-04-23 09:29:44,False,False
4,KumarSa61420257,Akhir bhojh ban gaye na bihari pure India pe...,0,0,0,2020-04-27 08:15:01,False,False


In [6]:
data_strip_none = df[df['text'].notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))
print(data_strip_none.shape)
data_strip_none.head()

(74553, 8)


,usernameTweet,text,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet
0,rj_ashwin,Sabse Pehle toh Majdur Labour Class Worker...,0,1,0,2020-03-28 23:24:57,False,False
1,vikramsoni232,TajinderBagga Bhaiya i proud of you You a...,0,0,0,2020-03-29 12:52:33,False,False
2,ABHISHE33329022,Mam India mein hum ko companys ki audit krni...,0,0,0,2020-03-29 21:20:30,True,False
3,hindust74474485,Videsh me rahne wale Bhai India laye Gaye bu...,0,0,0,2020-04-23 09:29:44,False,False
4,KumarSa61420257,Akhir bhojh ban gaye na bihari pure India pe...,0,0,0,2020-04-27 08:15:01,False,False


In [7]:
random.seed(9001)

data_api, data_test = train_test_split(data_strip_none, train_size=0.75, random_state=0)

# Reset index
data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
data_test = data_test.set_index(pd.Index(range(0,len(data_test))))

# Save the dataset
data_api.to_csv('dataApi.csv',index=False)
data_test.to_csv('dataTest.csv',index=False)

In [8]:
data_api = pd.read_csv('dataApi.csv')

In [10]:
authenticator = IAMAuthenticator(apikey)
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url(url)
tone_analyzer.set_disable_ssl_verification(False)

In [11]:
tones = []
index = 0
for item in data_api['text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [12]:
tones_copy = tones.copy()
tones = pd.DataFrame(tones)
tones.head()

,document_tone
0,"{'tones': [{'score': 0.664431, 'tone_id': 'sad..."
1,"{'tones': [{'score': 0.605127, 'tone_id': 'sad..."
2,"{'tones': [{'score': 0.689327, 'tone_id': 'joy..."
3,{'tones': []}
4,"{'tones': [{'score': 0.692069, 'tone_id': 'ana..."


In [15]:
data_api = data_api.reset_index()
data_api = data_api.drop(['index'], axis=1)
data_api = pd.concat([data_api5,tones],axis=1)

data_api.to_csv('data_api.csv',index=False)
dftest = pd.read_csv('data_api.csv')

In [16]:
dftest.head()

,usernameTweet,text,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,document_tone
0,SandeepKhasa_,The number of COVID19 patients are increasing...,0,0,0,2020-05-11 15:59:29,False,False,"{'tones': [{'score': 0.664431, 'tone_id': 'sad..."
1,katherinettyson,COVID19 has exposed that “populists” don’t ca...,0,2,0,2020-05-17 22:05:49,False,False,"{'tones': [{'score': 0.605127, 'tone_id': 'sad..."
2,AmBeautifulShow,Your emergency financial needs have been an im...,0,0,0,2020-05-12 20:42:23,True,False,"{'tones': [{'score': 0.689327, 'tone_id': 'joy..."
3,Bea_L_titanji,Covid19 compounding our already dire situati...,1,2,0,2020-05-18 23:00:04,False,False,{'tones': []}
4,MilesWebCA,Since all are at home due to COVID19 lockdown ...,0,0,0,2020-05-11 21:01:00,False,False,"{'tones': [{'score': 0.692069, 'tone_id': 'ana..."


In [17]:
dftest['document_tones'] = dftest.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
dftest.drop('document_tone',axis = 1,inplace = True)

In [18]:
# split sentiments in corresponding columns
for i in range(0, len(dftest)):
    for j in range(0, len(dftest.loc[i, 'document_tones'])):
        dic = dftest.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        dftest.loc[i, source] = 1
dftest.head()

,usernameTweet,text,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,document_tones,sadness,joy,analytical,confident,tentative,anger,fear
0,SandeepKhasa_,The number of COVID19 patients are increasing...,0,0,0,2020-05-11 15:59:29,False,False,"[{'score': 0.664431, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,katherinettyson,COVID19 has exposed that “populists” don’t ca...,0,2,0,2020-05-17 22:05:49,False,False,"[{'score': 0.605127, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,AmBeautifulShow,Your emergency financial needs have been an im...,0,0,0,2020-05-12 20:42:23,True,False,"[{'score': 0.689327, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,Bea_L_titanji,Covid19 compounding our already dire situati...,1,2,0,2020-05-18 23:00:04,False,False,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MilesWebCA,Since all are at home due to COVID19 lockdown ...,0,0,0,2020-05-11 21:01:00,False,False,"[{'score': 0.692069, 'tone_id': 'analytical', ...",NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [19]:
dftest.to_csv('ToneAnalyzer6.csv')